In [ ]:
import torch.optim as optim
import torch.nn as nn
import torch
from torch.utils.data import DataLoader
from utils.train_audio import train_model
from utils.dataset_audio import AudioDataset

if __name__ == "__main__":
    data_dir = './data'
    batch_size = 32
    img_size = 224
    num_epochs = 50
    learning_rate = 0.001
    patience = 5
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    torch.manual_seed(42)

    train_dataset = AudioDataset(data_dir, split="train", sample_rate=16000, duration=3, transform=None, shuffle=True)
    # val_dataset = AudioDataset(data_dir, split="val", img_size=img_size)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    # val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
